In [1]:
# 열린 크롬창 내에 csv 버튼까지 보이게끔 해둬야 코드가 실행됨.

# 밑에 다운로드 한 이력이 있으면 안돌아감.

# 크롤링을 하려면 selenium, chromedriver, urllib이 필수적으로 필요하다.

chromedriver의 경우 Only local connections are allowed. 라는 이슈가 있기에, 서버에서는 사용 불가능하다. <br>
그리고 일반 크롬과 버전이 맞아야 사용 가능하므로, 사용전에 버전 확인을 하는 것이 좋다. <br>
따로 버전 확인을 하지 않더라도 에러가 다 알려주니 크게 신경쓰지 않아도 됨. <br>

[chromedriver.exe 다운받는 경로](https://chromedriver.chromium.org/downloads) <br>

(현재 주피터를 실행하는 경로에) 현재 내 pc의 chrome과 버전이 동일한 chromedriver.exe가 깔려만 있으면 잘 실행됨.

In [2]:
# pip install --upgrade [selenium]

In [3]:
import sys, os
import requests #(네이버) api 이용할 때 쓰는 패키지
import selenium #크롤링 패키지

# selenium의 webdriver를 사용하기 위한 import
from selenium import webdriver

# selenium으로 무엇인가 입력하기 위한 import
from selenium.webdriver.common.keys import Keys
#

from pandas import DataFrame
from bs4 import BeautifulSoup #크롤링 패키지
import re #정규식 표현 모듈
from datetime import datetime
import pickle, progressbar, json, glob, time #pickle: 객체를 파일로 저장해주는 모듈, progressbar: 진행표시 바, json: 네이버 api, glob: 파일들 리스트 뽑을 때 사용
# https://wikidocs.net/83
from tqdm import tqdm #진행표시바
#

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

import numpy as np

import urllib.request #파싱을 위한 모듈

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 크롬창 열기

In [94]:
print('브라우저를 실행시킵니다(자동 제어)\n')
browser = webdriver.Chrome('chromedriver') #크롬창열기 # (현재 주피터를 실행하는 경로에) 현재 내 크롬 버전과 동일한 chromedriver.exe가 있으면 실행됨.
# https://beomi.github.io/2017/09/28/HowToMakeWebCrawler-Headless-Chrome/

news_url = 'https://www.kdca.go.kr/npt/biz/npp/ist/bass/bassSexdstnAgeStatsMain.do'

browser.get(news_url) #url 오픈
time.sleep(2)

# version issue
# SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 87
# Current browser version is 103.0.5060.114 with binary path C:\Program Files (x86)\Google\Chrome\Application\chrome.exe

# solution
# https://codechacha.com/ko/selenium-chromedriver-version-error/

브라우저를 실행시킵니다(자동 제어)



In [5]:
시도lst = ['서울', '부산', '대구', '인천',
      '광주', '대전', '울산', '경기', '강원', 
      '충북', '충남', '전북', '전남', '경북',
      '경남', '제주', '세종']
len(시도lst) # 17

17

In [117]:
# 윤년list (4년에 한 번씩)
윤년list = [2020 - i*4 for i in range(5)]


# 날짜list
시작lst = []
끝lst = []
for 연도 in range(2001, 2022):
    for 월 in range(1, 13):
        if 월//10 < 1:
            월 = f'0{월}'
        시작lst.append(f'{연도}-{월}-0{1}')
        
        if 월 in ['01', '03', '05', '07', '08', 10, 12]:
            끝lst.append(f'{연도}-{월}-{31}')
            
        elif 월 in ['04', '06', '09', 11]:
            끝lst.append(f'{연도}-{월}-{30}')
                
        elif 월 == '02':
            if 연도 not in 윤년list:
                끝lst.append(f'{연도}-{월}-{28}')
            else:
                끝lst.append(f'{연도}-{월}-{29}')



## 머리가 아프니, 2022년은 따로 만들어서 붙이도록 한다.
for 월 in range(1, 7):
    if 월//10 < 1:
        월 = f'0{월}'
    시작lst.append(f'{2022}-{월}-0{1}')
    if 월 in ['01', '03', '05', '07', '08', 10, 12]:
        끝lst.append(f'{2022}-{월}-{31}')

    elif 월 in ['04', '06', '09', 11]:
        끝lst.append(f'{2022}-{월}-{30}')

    elif 월 == '02':
        if 2022 not in 윤년list:
            끝lst.append(f'{2022}-{월}-{28}')
        else:
            끝lst.append(f'{2022}-{월}-{29}')

In [118]:
시작lst[:5]
끝lst[:5]

len(시작lst)
len(끝lst)
## 258개여야함. (12*21 + 6)

['2001-01-01', '2001-02-01', '2001-03-01', '2001-04-01', '2001-05-01']

['2001-01-31', '2001-02-28', '2001-03-31', '2001-04-30', '2001-05-31']

258

258

# 멈춘 부분에서 다시 시작
- 마지막으로 적혀있는 시작날짜부터 시작하면 됨. <br>
- 2022-07-15, 12:42분 현재, (58)까지 저장됨. <br>
    2005-12-01 부터 다시 시작 <br>
    
- 12:57분 현재, (97)까지 저장됨. <br>
    2009-03-01부터 다시 시작 <br>

- 01:05분 현재, (... 이름이 시간 형태로 달라짐)까지 저장됨. <br>
    2011-02-01 부터 다시 시작 <br>

- 01:13분 현재, (... 이름이 시간 형태로 달라짐)까지 저장됨. <br>
    2013-01-01 부터 다시 시작 <br>
   
- 01:21분 현재, <br>
    2014-12-01 부터 다시 시작 <br>
- 01:33분 현재, <br>
    2018-10-01 부터 다시 시작 <br>

- 01:37분 현재, <br>
    2019-08-01 부터 다시 시작 해야함. <br>

- 01:45분 현재, <br>
    강남 끝남, 강동 시작해야함. <br>
    다시 처음날짜부터 시작해야하는데 마지막으로 멈춘부분부터 시작해버리는 이슈 발생, 다행히 파일저장까진 안함. <br>
    
----------------
    
- 01:57분 현재, <br>
    강동 2004-09-01부터 다시 시작 <br>
    
- 02:08분 현재, <br>
    강동 2008-09-01부터 다시 시작 <br>
    
- 02:12분 현재, <br>
    강동 2009-07-01부터 다시 시작 <br>
    
- 02:17분 현재, <br>
    강동 2011-07-01부터 다시 시작 <br>

- 02:20분 현재, <br>
    강동 2012-03-01부터 다시 시작 <br>

- 02:26분 현재, <br>
    강동 2013-03-01부터 다시 시작해야함. <br>


- 11:12분 현재, <br>
    강동 2015-12-01부터 다시 시작 <br>

- 11:25분 현재, <br>
    강동 2018-06-01부터 다시 시작 <br>
    
- 11:30분 현재, <br>
    강동 2019-03-01부터 다시 시작 <br>

- 11:33분 현재, <br>
    강동 2019-10-01부터 다시 시작 <br>

- 11:38분 현재, <br>
    강동 2020-06-01부터 다시 시작 <br>

- 11:42분 현재, <br>
    강동 2021-01-01부터 다시 시작 <br>
    에러 빈도수가 (8개월로) 잦아짐. 일단 강동은 끝내고 그 다음부터는 더 빨리 새로고침 하기로...

- 11:46분 현재, <br>
    강동 2021-09-01부터 다시 시작 <br>

- 11:48분 현재, <br>
    강동 2022-03-01부터 다시 시작 <br>
    
------------------

- 11:53분 현재, <br>
    강북 2001-07-01부터 다시 시작 해야함.

- 13:42분 현재, <br>
    강북 2004-02-01부터 다시 시작 해야함.

- 14:22분 현재, <br>
    강북 2018-09-01부터 다시 시작 해야함.
    
- 14:26분 현재, <br>
    강북 2019-04-01부터 다시 시작

- 14:26 현재, <br>
    강북 2019-11-01부터 다시 시작

- 14:32 현재, <br>
    강북 2020-06-01부터 다시 시작
    
- 14:36 현재, <br>
    강북 2021-01-01부터 다시 시작

- 14:41 현재, <br>
    강북 끝!
------------------

- 14:49 현재, <br>
    강서 2002-11-01 부터 다시 시작

- 14:52 <br>
    강서 2003-06-01부터
    
- 15:05 <br>
    강서 2007-04-01부터

- 15:13 <br>
    강서 2010-07-01부터

- 15:19 <br>
    강서 2012-05-01부터

- 15:44 <br>
    강서 2022-04-01부터

- 15:46 <br>
    강서 끝!
----------------------------
관악구 중

In [110]:
멈춘부분 = 시작lst.index('')
시작lst = 시작lst[멈춘부분:]
끝lst = 끝lst[멈춘부분:]

In [111]:
시작lst[0]
시작lst[-1]

'2022-04-01'

'2022-06-01'

# 옵션버튼 클릭

In [112]:
def delete(input_box):
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)
        input_box.send_keys(Keys.BACK_SPACE)

# 통계작성_다운로드, 매 loop마다 쓰이므로 함수로 작성해서 집어넣도록 하겠습니다.
def 통계작성_다운로드():
    ## 통계작성버튼 클릭
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/input[2]').click()
#     browser.implicitly_wait(2)
    time.sleep(5) # 여기 시간 늘린게 가장 큰 성공 요인인 것 같음.

    ## csv 다운로드 클릭 //*[@id="csvDownBtn"] //*[@id="csvDownBtn"]/img
    browser.find_element_by_xpath('//*[@id="csvDownBtn"]').click()
#     browser.implicitly_wait(3)
    time.sleep(5)

# 객체.implicitly_wait(시간)은 작동? 완료 되면 그냥 무시되는 코드

In [113]:
# browser.refresh()로 해결
# def 다시실행():
#     browser = webdriver.Chrome('chromedriver') #크롬창열기 # (현재 주피터를 실행하는 경로에) 현재 내 크롬 버전과 동일한 chromedriver.exe가 있으면 실행됨. # 반드시 같이 있어야 함.
#     browser.get(news_url) #url 오픈
#     time.sleep(2)
    

In [114]:
def 공통():
    ## 연령별 성별 탭 클릭
    search_opt_box = browser.find_element_by_xpath('//*[@id="ui-id-3"]')
    search_opt_box.click() #버튼 클릭
    browser.implicitly_wait(0.02)

    ## 공통
    # 감염병 명 선택
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[1]/div/button/div').click()
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[1]/div/div/ul/li[1]/label/input').click()
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/button/div').click()
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/div/ul/li[1]/label/input').click()
    browser.implicitly_wait(0.02)

    ## 끝에는 닫아주기까지 해야함.
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[2]/li[2]/div[2]/div/button/div').click()

    # 연령 선택
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm_ageType2"]').click()
    

## 창 새로 열기 전에 하던 시도군구부터 다시 시작해야하므로
def 시도군구(시도_, 군구_):
    ## 시도
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrmAreaCtprvnCd"]').click()
    browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaCtprvnCd"]/option[{시도_+2}]').click()
    ## 군구
    browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구_+2}]').click()

In [115]:
# 기간 선택, 함수로 만들어서 한 줄로 실행하는 코드를 집어넣도록 하겠습니다.
def 기간선택(시도_, 군구_):
    for idx in range(len(시작lst)):
        print(f'시작 날짜입니다. {시작lst[idx]}')
        
        # 8개월이 지나면 창 다시 실행.
        if idx != 0 and idx%8 == 0: # 이것도 에러 잡은 성공 요인.
#             다시실행()
            browser.refresh()
            공통()
            시도군구(시도_,군구_)
            

        ## 시작 날짜
        search_box = browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrmStartDt"]')
        delete(search_box)
        browser.implicitly_wait(0.1)
        search_box.send_keys(시작lst[idx])
        search_box.send_keys(Keys.RETURN)
        browser.implicitly_wait(0.1)

        ## 끝 날짜
        search_box = browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrmEndDt"]')
        delete(search_box)
        browser.implicitly_wait(0.1)
        search_box.send_keys(끝lst[idx])
        search_box.send_keys(Keys.RETURN)
        browser.implicitly_wait(0.1)
        browser.find_element_by_xpath('//*[@id="tabs-3"]').click() # 빈 화면 클릭하기.
        time.sleep(0.3)

        # 8개월되기 전(7개월째)의 정보 저장
        if idx%7 == 0:
            시도 = 시도_
            군구 = 군구_
        
        ## 닫아주기
#         browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrm"]/div/ul[1]/li[2]/img[2]').click()
        
        통계작성_다운로드()

In [116]:
공통()


# 지역 설정, 하나하나 클릭해야 하므로 Xpath의 패턴을 찾아서 for loop을 돌릴 수 있도록 한다.
## 시도
for 시도 in range(18):
    print(f'{시도lst[시도]}를(을) 돌리고 있습니다.')
    browser.find_element_by_xpath('//*[@id="sexdstnAgeAgeSexdstnFrmAreaCtprvnCd"]').click()
    browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaCtprvnCd"]/option[{시도+2}]').click()
        
    ## 군구
    if 시도lst[시도] == '경기':
        for 군구 in range(52): # 52
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            기간선택(시도, 군구)


    elif 시도lst[시도] == '서울' or 시도lst[시도] == '경북' or 시도lst[시도] == '경남':
        for 군구 in range(4, 25): # 25 # 강남, 강동, 강북, 강서 끝나서 그 다음부터!!
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            기간선택(시도, 군구)
#             break
#         break
#     break


    elif 시도lst[시도] == '전남':
        for 군구 in range(22): # 22
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            기간선택(시도, 군구)


    elif 시도lst[시도] == '강원' or 시도lst[시도] == '충남':
        for 군구 in range(18): # 18
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            기간선택(시도, 군구)
            

    elif 시도lst[시도] == '충북':
        for 군구 in range(17): # 17
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            기간선택(시도, 군구)
            

    elif 시도lst[시도] == '부산' or 시도lst[시도] == '전북':
        for 군구 in range(16): # 16
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            기간선택(시도, 군구)


    elif 시도lst[시도] == '인천':
        for 군구 in range(10): # 10
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            기간선택(시도, 군구)


    elif 시도lst[시도] == '대구':
        for 군구 in range(8): # 8
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            기간선택(시도, 군구)


    elif 시도lst[시도] == '광주' or 시도lst[시도] == '대전' or 시도lst[시도] == '울산':
        for 군구 in range(5): # 5
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            기간선택(시도, 군구)


    elif 시도lst[시도] == '제주':
        for 군구 in range(4): # 4
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            기간선택(시도, 군구)


    elif 시도lst[시도] == '세종':
        for 군구 in range(1): # 1
            browser.find_element_by_xpath(f'//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[{군구+2}]').click()
            기간선택(시도, 군구)
            
    time.sleep(0.4)

서울를(을) 돌리고 있습니다.
시작 날짜입니다. 2022-04-01
시작 날짜입니다. 2022-05-01
시작 날짜입니다. 2022-06-01


KeyboardInterrupt: 

# Message: element click intercepted
이 문제는 새로고침으로 해결.... 도랏맨..... 진짜 미쳤나봐 <br>
또는 창을 최대화하고, 그 눌려야하는 버튼이 창에 보이게끔 화면 축소를 하거나.... <br>
또는 다운 받은 내역이 뜬 창을 닫거나 <br>

또 다른 에러가 뜨면 코드를 재실행 하면 문제 없음.ㅎ_ㅎ <br>

일단 한 번 돌리면 최대 5년 그냥 돌아감. 아싸. 난 우영우 보러 가야지 ^^ <br>

(2년으로 줄어듦...) <br>

no search element 라는 에러도 걍 무시하고 다시 실행해주면 됨...^^....

In [ ]:
53-2+1

<각 지역 군구>
~~서울 : 1~25 ==> 25~~
~~부산 : 26~41 ==> 16~~
~~대구 : 42~49 ==> 8~~
~~인천 : 50~59 ==> 10~~

광주 : 60~64 ==> 5
대전 : 65~69 ==> 5
울산 : 70~74 ==> 5

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[2]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[47]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[48]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[53]
~~경기 : 75~120, 120(이게 2개임) 122, 123, 124, 125, 130 ==> 52 (2로 시작, 53으로 끝)~~

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[2]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[19]
~~강원 : 360~377 ==> 18 (2로 시작, 19로 끝)~~

~~충북 : 400~416 ==> 17~~

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[2]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[19]
~~충남 : 420~435 + 704, 705 ==> 18~~

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[2]
//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[17]
~~전북 : 550~565 ==> 16~~

~~전남 : 570~591 ==> 22~~

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[26]
~~경북 : 600~624 ==> 25~~

//*[@id="sexdstnAgeAgeSexdstnFrmAreaSignguCd"]/option[26]
~~경남 : 650~669, 706~710 ==>25~~

~~제주 : 700~703 ==> 4~~
~~세종 : 711 ==> 1~~

In [ ]:
for 군구 in range(1): # 1
    print(군구)

In [5]:
len(시작lst[:시작lst.index('2013-03-01')]) # 내가 해냄,,,

146

In [6]:
len(시작lst)

258